# Introduction to Portfolio Theory: Risk Measures #

In [24]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Statistics
from scipy.stats import norm

# Import Data
import yfinance as yf

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_financial_data

# Visualization
import matplotlib.pyplot as plt

In [25]:
# Import Data

df_amzn = import_financial_data("AMZN", starting_year='2020')
df_nvda = import_financial_data("NVDA", starting_year='2020')
df_tsla = import_financial_data("TSLA", starting_year='2020')

# Create the data we are going to use
data = pd.DataFrame()

data['AMZN'] = df_amzn['adj_close']
data['NVDA'] = df_nvda['adj_close']
data['TSLA'] = df_tsla['adj_close']

data

In [26]:
# Calculate Logarithmic Returns

log_returns = np.log(data / data.shift(1)).dropna()

log_returns

In [27]:
# 1. Annualized Volatility (Standard Deviation)

volatility = log_returns.std() * np.sqrt(252)

print("Annualized Volatility:")
print(volatility)

In [28]:
# 1.5 Annualized Variance (Standard Deviation)

variance = log_returns.var() * 252

print("Anualized Variance:")
print(variance)

In [29]:
# 2. Covariance Matrix
covariance = log_returns.cov() * 252  # Anualized

print("\nCovariance Matrix:")
print(covariance)

In [30]:
# 3. Correlations
correlations = log_returns.corr()

print("\nCorrelation Matrix:")
print(correlations)

In [31]:
# 4. Value at Risk

confidence_level = 0.95

z_score = norm.ppf(1 - confidence_level)

VaR_95 = log_returns.mean() + z_score * log_returns.std()
VaR_95 = VaR_95 * np.sqrt(252)  # Anualized
print("\nAnualized Value at Risk (VaR) at 95% confidence level:")
print(VaR_95)


In [32]:
# Download the SPY (benchmark)
# S&P500
benchmark = yf.download('^GSPC', start='2020-01-01', end='2025-01-01', interval='1d', auto_adjust=True)
benchmark.dropna(inplace=True)

benchmark

In [33]:
# Calculate the benchmark returns

benchmark_rets = np.log(benchmark['Close'] / benchmark['Close'].shift(1)).dropna()

# Reindex
benchmark_rets = benchmark_rets.reindex(log_returns.index)

benchmark_rets

In [34]:
# Calculate the Excess Returns

excess_return = log_returns.subtract(benchmark_rets.squeeze(), axis=0)

excess_return

In [35]:
# Now calculate the Tracking Error (which is the Std of the Excess Returns)

tracking_error = excess_return.std() * np.sqrt(252)

print("\nTracking Error:")
print(tracking_error)

In [36]:
#Let us visualize the Risk Measures

fig, ax = plt.subplots(figsize=(10, 6))
volatility.plot(kind='bar', color='skyblue', label='Volatility (Risk)', alpha=0.5)
tracking_error.plot(kind='bar', color='lightgreen', label='Tracking Error', alpha=0.5)
VaR_95.plot(kind='bar', color='salmon', label='VaR 95%', alpha=0.7)
plt.title('Risk Measures')
plt.ylabel('Value')
plt.legend()
plt.show()

In [37]:
# Tesla seems more riskier than Amazon

mean = log_returns.mean() * 252

print("Annualized Mean Returns:")
print(mean)

In [38]:
# Risk Adjusted Returns

eff_ratio = mean/volatility

print("Risk Adjusted Returns:")
print(eff_ratio)

In [39]:
#Let us visualize

fig, ax = plt.subplots(figsize=(10, 6))
eff_ratio.plot(kind='bar', color='lightpink', label='Risk Adjusted Returns')
ax.axhline(y=1, color='black', linestyle='--')
plt.title('Risk Adjusted Returns')
plt.ylabel('Value')
plt.legend()
plt.show()

In [42]:
# Calculate the Risk Adjusted Returns of the Benchmark (Risk Aversion Coefficient)

mean_bench = benchmark_rets.mean() * 252

var_bench = benchmark_rets.var() * 252

eff_rate_bench = mean_bench/var_bench

eff_rate_bench.values[0]